In [2]:
import glob
from pathlib import Path
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

# Utils

In [18]:
def lower_text(text):
    return text.lower()
def hard_remove_spaces(text):
    return " ".join(text.split())

lowerText = lambda x: lower_text(x)
hardRemoveSpaces = lambda x: hard_remove_spaces(x)

# Aggregate

In [ ]:
file_ = glob.glob("../sites_data/*")[0]
df=pd.read_csv(file_,index_col=None)#.drop('Unnamed: 0', 1)
df['source'] = Path(file_).stem.split('-')[0]
print(Path(file_).stem)

for file_ in glob.glob("../sites_data/*")[1:]:
    print(Path(file_).stem)
    df_file = pd.read_csv(file_,index_col=None)
    df_file['source'] = Path(file_).stem.split('-')[0]
    df=pd.concat([df,df_file],axis=0).reset_index(drop=True)

In [ ]:
print(df.columns)

In [ ]:
selected_columns = ['url', 'title', 'salary', 'experiment', 'degree', 'so_luong',
       'industry', 'location', 'level', 'jd', 'benefit', 'requirement',
       'source', ]

df = df[selected_columns]

print(f'Number of jobs post: {df.shape[0]}')
df['source'].value_counts()

# Language detect

In [ ]:
df['language'] = pd.Series(0, index=df.index)

from langdetect import detect
for i in range(df.shape[0]):
    # REINITIALIZE THE API
    try:        
        df['language'][i] = detect(str(df.requirement[i]))
    except:
        continue
    if i%10000==0:         print(i)

In [ ]:
df['language'].value_counts().to_frame()

## `vi` and `en`

In [ ]:
df = df.loc[(df.language == 'vi') | (df.language == 'en')].reset_index(drop=True)#.language.value_counts()

In [ ]:
df['source'].value_counts().to_frame()

In [153]:
df[~df['degree'].isnull()]['source'].value_counts()#.reset_index(drop=True).astype(str)


jobsGo            18397
sieuthivieclam    13964
vieclam24h         1102
timviecnhanh        980
Name: source, dtype: int64

# blue_or_white
- Clean up degree
- Blue:
    - Site that blue in nature
        - tuyencongnhan
    - `degree` is in `blue_degree_list`

In [ ]:
df['blue_or_white'] = 'white'

## Site that blue in nature
- tuyencongnhan

In [ ]:
df.loc[df.source=='tuyencongnhan','blue_or_white'] = 'blue'

## By degree

### Clean up degree

In [ ]:
df.loc[~df['degree'].isnull(),'degree'] = df.loc[~df['degree'].isnull(),'degree'].apply(hardRemoveSpaces)
df.loc[~df['degree'].isnull(),'degree'] = df.loc[~df['degree'].isnull(),'degree'].apply(lowerText)

In [ ]:
df['degree'].value_counts()
# df

### Assigning blue tag

In [ ]:
blue_degree_list = ['trung cấp', 'trung học',  'không yêu cầu',
                    'chứng chỉ', 'trung học phổ thông','trung cấp - nghề', 
                    'chứng chỉ chuyên ngành', 'lao động phổ thông', 'chứng chỉ - sơ cấp nghề',
                    ]
df.loc[df['degree'].isin(blue_degree_list),'blue_or_white'] = 'blue'

### English jobs are white

In [ ]:
df.loc[df['blue_or_white']=='blue','language'].value_counts()

# df.loc[df.language=='en','blue_or_white'] = 'white'

# Clean up requirement

### requirement to list of requirements

In [5]:
import re
def string_to_list(text):
    string_list = re.split('\n|\t|•|-',text)
    string_list = [s.strip() for s in string_list]
    # string_list = [s.lower() for s in string_list]
    return list(filter(None, string_list))

stringToList = lambda x: string_to_list(x)

In [5]:
df['requirement_list'] = df.requirement.apply(stringToList)

### Clean requirement_list

In [8]:
def scrub_words(text):
    """Basic cleaning of texts."""
    
    # remove html markup
    text=re.sub("(<.*?>)"," ",text)
    
    #remove non-ascii and digits
    text=re.sub("(\\W|\\d)"," ",text)
    
    #remove whitespace
    text = " ".join(text.split())

    text = text.lower()

    return text

def scrub_words_list(text_list):
    cleaned_text_list = []
    for text in text_list:
        cleaned_text_list.append(scrub_words(text))
    return list(filter(None, cleaned_text_list))

scrubWords = lambda x: scrub_words(x)
scrubWordsList = lambda x: scrub_words_list(x)

In [7]:
df['requirement_list_cleaned'] = df.requirement_list.apply(scrubWordsList)

## Tokenize requirements

### Vietnamese

In [11]:
from underthesea import word_tokenize #as word_tokenize_vi

def word_tokenize_from_text_vi(text):
    return word_tokenize(text)

def word_tokenize_from_text_vi_2(text):
    return word_tokenize(text,format="text").split()

def word_tokenize_from_list_vi(list_of_text):
    words = []
    for text in list_of_text:
        words += word_tokenize_from_text_vi(text)
    return words

def word_tokenize_from_list_vi_2(list_of_text):
    words = []
    for text in list_of_text:
        words+=word_tokenize_from_text_vi_2(text)
    return words

tokenizeListVi = lambda x:word_tokenize_from_list_vi(x)
tokenizeListVi2 = lambda x:word_tokenize_from_list_vi_2(x)

In [9]:
df['tokenized_requirement'] = np.NaN

In [10]:
df.loc[df.language == 'vi','tokenized_requirement'] = df.loc[df.language == 'vi','requirement_list_cleaned'].apply(tokenizeListVi)

In [12]:
df.loc[df.language == 'vi','tokenized_requirement_2'] = df.loc[df.language == 'vi','requirement_list_cleaned'].apply(tokenizeListVi2)

### English

Now tokenizing English is harsh:
- remove step words
- stem
- so tokenizing basically break string into words
May consider other methods

#### Clean english a bit more

In [13]:
import nltk
from nltk.stem import PorterStemmer

In [14]:
def remove_stop_from_string(text):
    stopwords = nltk.corpus.stopwords.words('english')
    return ' '.join([word for word in text.split() if word not in stopwords])
    
def remove_stop_from_string_for_list(list_of_text):
    cleaned_list = []
    for text in list_of_text:
        cleaned_list.append(remove_stop_from_string(text))
    return cleaned_list

removeStopforListEn = lambda x: remove_stop_from_string_for_list(x)

def porter_stemm(text):
    porter_stemmer=PorterStemmer()
    stemmed_words=[porter_stemmer.stem(word=word) for word in text.split()]
# stemmed_words = set(stemmed_words)
    return ' '.join(stemmed_words)

def porter_stemm_for_list(list_of_text):
    stemmed_list = []
    for text in list_of_text:
        stemmed_list.append(porter_stemm(text))
    return stemmed_list

porterStemmingListEn = lambda x:porter_stemm_for_list(x)

In [15]:
df.loc[df.language == 'en','requirement_list_cleaned'] = df.loc[df.language == 'en','requirement_list_cleaned'].apply(removeStopforListEn)

df.loc[df.language == 'en','requirement_list_cleaned'] = df.loc[df.language == 'en','requirement_list_cleaned'].apply(porterStemmingListEn)

In [16]:
from nltk.tokenize import word_tokenize

def word_tokenize_from_text_en(text):
    return word_tokenize(text)

def word_tokenize_from_list_en(list_of_text):
    words = []
    for text in list_of_text:
        words += word_tokenize_from_text_en(text)
    return words

tokenizeListEn = lambda x:word_tokenize_from_list_en(x)

In [17]:
df.loc[df.language == 'en','tokenized_requirement'] = df.loc[df.language == 'en','requirement_list_cleaned'].apply(tokenizeListEn)

# Cleanup industry
Each side is a different mess

In [20]:
import re
def string_to_list(text):
    string_list = re.split('\n|\t|•|-|,',text)
    string_list = [s.strip() for s in string_list]
    # string_list = [s.lower() for s in string_list]
    return list(filter(None, string_list))

stringToList = lambda x: string_to_list(x)

df.loc[df.source == 'vietnamworks', 'industry'] = df.loc[df.source == 'vietnamworks', 'industry'].apply(stringToList)

In [21]:
def string_to_list(text):
    text=re.sub("IT","It",text)
    text=re.sub("PR","Pr",text)
    text=re.sub("-","",text)
    text=" ".join(text.split())
    string_list = re.findall('[A-Z][^A-Z]*', text)
    # string_list = [re.split('-',t) for t in string_list]

    return list(filter(None, string_list))

stringToList = lambda x: string_to_list(x)

df.loc[df.source == 'vieclam24h', 'industry'] = df.loc[df.source == 'vieclam24h', 'industry'].apply(stringToList)

In [22]:
def string_to_list(text):
    string_list = re.split('\n|\t|•|-|,',text)
    string_list = [s.strip() for s in string_list]
    # string_list = [s.lower() for s in string_list]
    return list(filter(None, string_list))[:-1]

stringToList = lambda x: string_to_list(x)
df.loc[df.source == 'jobsGo', 'industry'] = df.loc[df.source == 'jobsGo', 'industry'].apply(stringToList)

In [23]:
def string_to_list(text):
    string_list = re.split('\n',text)
    string_list = [s.strip() for s in string_list]
    # string_list = [s.lower() for s in string_list]
    return list(filter(None, string_list))

stringToList = lambda x: string_to_list(x)

df.loc[df.source == 'tuyencongnhan', 'industry'] = df.loc[df.source == 'tuyencongnhan', 'industry'].apply(stringToList)

In [24]:
def string_to_list(text):
    string_list = re.split('\n|/',text)
    string_list = [s.strip() for s in string_list]
    # string_list = [s.lower() for s in string_list]
    return list(filter(None, string_list))

stringToList = lambda x: string_to_list(x)

df.loc[df.source == 'topCV', 'industry'] = df.loc[df.source == 'topCV', 'industry'].apply(stringToList)

In [25]:
def string_to_list(text):
    string_list = re.split(',|/',text)
    string_list = [s.strip() for s in string_list]
    # string_list = [s.lower() for s in string_list]
    return list(filter(None, string_list))

stringToList = lambda x: string_to_list(x)

df.loc[df.source == 'careerBuilder', 'industry']  = df.loc[df.source == 'careerBuilder', 'industry'].apply(stringToList)

In [26]:
def string_to_list(text):
    # string_list = re.split(',|/',text)
    # string_list = [s.strip() for s in string_list]
    # string_list = [s.lower() for s in string_list]
    text=re.sub("/","",text)
    text=re.sub(" - ","",text)
    string_list = re.findall('[A-Z][^A-Z]*', text)
    return list(filter(None, string_list))

stringToList = lambda x: string_to_list(x)

df.loc[df.source == 'timviecnhanh', 'industry']  = df.loc[df.source == 'timviecnhanh', 'industry'].apply(stringToList)

In [27]:
def string_to_list(text):
    string_list = re.split('\n|-',text)
    string_list = [s.strip() for s in string_list]
    # string_list = [s.lower() for s in string_list]
    return list(filter(None, string_list))

stringToList = lambda x: string_to_list(x)

df.loc[df.source == 'sieuthivieclam', 'industry'] = df.loc[df.source == 'sieuthivieclam', 'industry'].apply(stringToList)

In [28]:
def lower_list(li):
    return [x.lower() for x in li]

lowerList = lambda x: lower_list(x)

In [30]:
df.industry = df.industry.apply(lowerList)

# Cleanup JD

In [6]:
df['jd_list'] = df.jd.apply(stringToList)

In [9]:
df['jd_list_cleaned'] = df.jd_list.apply(scrubWordsList)

In [13]:
df['tokenized_jd_2'] = np.NaN

df.loc[df.language == 'vi','tokenized_jd_2'] = df.loc[df.language == 'vi','jd_list_cleaned'].apply(tokenizeListVi2)

In [16]:
df.head(3)

,url,title,salary,experiment,degree,so_luong,industry,location,level,jd,...,source,language,blue_or_white,requirement_list,requirement_list_cleaned,tokenized_requirement,tokenized_requirement_2,jd_list,jd_list_cleaned,tokenized_jd_2
0,https://vieclam24h.vn/nhan-vien-kinh-doanh/nha...,Nhân Viên Kinh Doanh Lĩnh Vực Chuyển Phát Nhan...,10 - 40 triệu,Chưa có kinh nghiệm,cao đẳng,5.0,"[quản trị kinh doanh, ngoại thương , xuất nhập...",Hà Nội,Chuyên viên- nhân viên,"- Chủ động tìm kiếm thông tin khách hàng mới, ...",...,vieclam24h,vi,white,[Ưu tiên có kinh nghiệm làm trong lĩnh vực gia...,[ưu tiên có kinh nghiệm làm trong lĩnh vực gia...,"[ưu tiên, có, kinh nghiệm, làm, trong, lĩnh vự...","[ưu_tiên, có, kinh_nghiệm, làm, trong, lĩnh_vự...","[Chủ động tìm kiếm thông tin khách hàng mới, t...",[chủ động tìm kiếm thông tin khách hàng mới th...,"[chủ_động, tìm_kiếm, thông_tin, khách_hàng, mớ..."
1,https://vieclam24h.vn/ban-hang/nhan-vien-ban-h...,Nhân Viên Bán Hàng Tại Showroom Thu Nhập Trên ...,15 - 20 triệu,Dưới 1 năm,trung cấp,3.0,"[bán hàng, nhân viên kinh doanh, tư vấn]",Hà Nội,Chuyên viên- nhân viên,"- Làm việc tại Showroom BigC The Garden, Mễ...",...,vieclam24h,vi,blue,"[Có kinh nghiệm về bán hàng, yêu thích bán hàn...",[có kinh nghiệm về bán hàng yêu thích bán hàng...,"[có, kinh nghiệm, về, bán, hàng, yêu thích, bá...","[có, kinh_nghiệm, về, bán, hàng, yêu_thích, bá...","[Làm việc tại Showroom BigC The Garden, Mễ ...",[la m viê c ta i showroom bigc the garden mễ t...,"[la, m, viê, c, ta, i, showroom, bigc, the, ga..."
2,https://vieclam24h.vn/dich-vu/shopee-nhan-vien...,Shopee - Nhân Viên Chăm Sóc Khách Hàng,5 - 7 triệu,Dưới 1 năm,trung học,15.0,"[dịch vụ, thương mại điện tử, tư vấn]",TP.HCM,Chuyên viên- nhân viên,MÔ TẢ CÔNG VIỆC- Tiếp nhận và giải đáp thắc mắ...,...,vieclam24h,vi,blue,"[Tốt nghiệp THPT trở lên., Tuổi từ 18 đến 32.,...","[tốt nghiệp thpt trở lên, tuổi từ đến, tối thi...","[tốt nghiệp, thpt, trở, lên, tuổi, từ, đến, tố...","[tốt_nghiệp, thpt, trở, lên, tuổi, từ, đến, tố...","[MÔ TẢ CÔNG VIỆC, Tiếp nhận và giải đáp thắc m...","[mô tả công việc, tiếp nhận và giải đáp thắc m...","[mô_tả, công_việc, tiếp_nhận, và, giải_đáp, th..."


In [5]:
df.columns

Index(['url', 'title', 'salary', 'experiment', 'degree', 'so_luong',
       'industry', 'location', 'level', 'jd', 'benefit', 'requirement',
       'source', 'language', 'blue_or_white', 'requirement_list',
       'requirement_list_cleaned', 'tokenized_requirement',
       'tokenized_requirement_2'],
      dtype='object')

# Results
after aggregate, clean, ... 1 job post would has

- **job requirement**: list of tokenized words
    - For Vietnamese, 2 ways of tokenizing by [Underthesea](https://github.com/undertheseanlp/underthesea)
        - tokenized_requirement: word_tokenize(sentence): ['Chàng trai', '9X', 'Quảng Trị']
        - tokenized_requirement_2: word_tokenize(sentence, format="text"): ['Chàng_trai 9X Quảng_Trị]
    - For English: Now tokenizing English is harsh:
        - remove step words
        - stem
        - so tokenizing basically break string into words
        - May consider other methods

- **industry**: list of industries

- **JD**
    - For Vietnamese
        - tokenized_jd_2: same way as tokenized_requirement_2

In [15]:
import datetime
date_object = datetime.date.today()

df.to_hdf(f'aggregated-{date_object}.h5', key='df', mode='w')